In [2]:
import torch
import torch.nn as nn
from torch.distributions.categorical import Categorical
from tqdm import tqdm
from datetime import datetime
import matplotlib.pyplot as pp
import numpy as np
from ipywidgets import interact, IntSlider, fixed

from sigma_graph.envs.figure8.action_lookup import MOVE_LOOKUP, TURN_90_LOOKUP
from sigma_graph.envs.figure8.default_setup import OBS_TOKEN
from sigma_graph.envs.figure8.figure8_squad_rllib import Figure8SquadRLLib
from sigma_graph.envs.figure8.gflow_figure8_squad import GlowFigure8Squad
#from graph_scout.envs.base import ScoutMissionStdRLLib
import sigma_graph.envs.figure8.default_setup as default_setup
from sigma_graph.data.file_manager import check_dir, find_file_in_dir, load_graph_files
import model  # THIS NEEDS TO BE HERE IN ORDER TO RUN __init__.py!
import model.utils as utils
import model.gnn_gflow 
from trajectory import Trajectory
import losses

import torch.optim as optim
import wandb
import json
import random
import networkx as nx
import matplotlib.pyplot as plt

2024-05-20 13:41:15,541	WARNING deprecation.py:47 -- DeprecationWarning: `ray.rllib.agents.dqn` has been deprecated. Use `ray.rllib.algorithms.[dqn|simple_q|r2d2|apex_dqn]` instead. This will raise an error in the future!


In [3]:
gflowfigure8 = torch.load('models/gnn_10_steps.pt')

In [3]:
# TODO: there has to be a better way to do this
def norm_dict(unnorm):
    min_val = 9999999
    for key in unnorm:
        min_val = min(unnorm[key], min_val)
    for key in unnorm:
        unnorm[key] = unnorm[key] + (min(min_val, 0) * -1)
    sum = 0
    for key in unnorm:
        sum += unnorm[key]
    for key in unnorm:
        unnorm[key] /= sum
        
gflowfigure8._reset_agents()
state_dirs = {}
for node in range(27):
    states = [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
    states[0][node] = 1
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    state = torch.tensor(np.array([states[0],], dtype=np.int8)).to(device)
    probs = gflowfigure8.sampler.forward(state, [1])
    # (forward_prob, action) = gflowfigure8.probs_to_action(probs)
    total_probs = {}
    total_probs["NOOP"] = (probs[0]+probs[5]+probs[10]).tolist()
    total_probs["N"] = (probs[1]+probs[6]+probs[11]).tolist()
    total_probs["S"] = (probs[2]+probs[7]+probs[12]).tolist()
    total_probs["W"] = (probs[3]+probs[8]+probs[13]).tolist()
    total_probs["E"] = (probs[4]+probs[9]+probs[14]).tolist()
    #norm_dict(total_probs)

    state_dirs[node] = total_probs

print(state_dirs)
flows = state_dirs
print(f'flows {flows}')
map_info, _ = load_graph_files(map_lookup="S")

fig = plt.figure()
fig.patch.set_alpha(0.)
fig.tight_layout()

plt.axis('off')

col_map = ["gold"] * len(map_info.n_info)

nx.draw_networkx(map_info.g_acs, map_info.n_info, node_color=col_map, edge_color="blue", arrows=True)

# max_x = max(pos[0] for pos in map_info.n_info.values())
# max_y = max(pos[1] for pos in map_info.n_info.values())
# scale_factor = max(max_x, max_y) / 2000
# op_to_dir = {
#     'N': (0, 1), 
#     'S': (0, -1), 
#     'W': (-1, 0),
#     'E': (1, 0)
# }
# for node in map_info.g_acs.nodes:
#     pos = map_info.n_info[node]
#     x, y = pos
#     dir_flows = flows[node-1]

#     color_vals = np.linspace(0, 1.75, 100)
#     colors = [(color, 0, 0, 0) for color in color_vals]

#     for op in ['N', 'S', 'W', 'E']:
#         flow = int((dir_flows[op]) * 99)
#         dx, dy = op_to_dir[op]
#         plt.arrow(x, y, dx*scale_factor, dy*scale_factor, color=colors[flow], alpha=1, width=0.1, head_width=1)

#     circle_radius = 1.00 
#     circle_center_x = x + circle_radius  
#     circle_center_y = y + circle_radius 

#     flow = int((dir_flows['NOOP']) * 99)

#     circle = plt.Circle((circle_center_x, circle_center_y), circle_radius, color=colors[flow], alpha=0.8, fill=False, linewidth=2)

#     plt.gca().add_patch(circle)


#plt.close()

AttributeError: 'GlowFigure8Squad' object has no attribute 'sampler'

In [5]:
gflowfigure8.reset()
red_path = []
blue_path = []
step_rewards = []
total_reward = 0
gflowfigure8.reset_state()
for r in range(34):   
    step = gflowfigure8.step_fcn_coordinate_time(0)
    red_path.append(step['red_node'])
    blue_path.append(step['blue_node'])
    step_rewards.append(step['step_reward'])
    total_reward += step['step_reward']

print(f'step_rewards {total_reward}')
clipped_reward = torch.log(torch.tensor(total_reward)).clip(-20)
print(f'clipped_reward {clipped_reward}')

map_info, _ = load_graph_files(map_lookup="S")
col_map = ["gold"] * len(map_info.n_info)

def display_graph(index):
    fig, ax = plt.subplots(figsize=(8, 6))
    cur_col_map = col_map[:]
    cur_col_map[red_path[index]-1] = "red"
    cur_col_map[blue_path[index]-1] = "blue"
    nx.draw_networkx(map_info.g_acs, pos=map_info.n_info, node_color=cur_col_map, edge_color="blue", arrows=True, ax=ax)
    ax.set_title(f"step rewards {step_rewards[index]}")
    plt.axis('off')
    plt.show()

# Create an interactive widget to display different graphs
slider = IntSlider(min=0, max=33, step=1, value=0, description='Graph Index')
interact(display_graph, index=slider)

AttributeError: 'GlowFigure8Squad' object has no attribute 'total_reward'

In [4]:
# visualize fcn coordinate time

gflowfigure8.reset()
red_path = []
blue_path = []
step_rewards = []
total_reward = 0
gflowfigure8.reset_state()
for r in range(34):   
    step = gflowfigure8.step_fcn_coordinate_time(0)
    red_path.append(step['red_node'])
    blue_path.append(step['blue_node'])
    step_rewards.append(step['step_reward'])
    total_reward += step['step_reward']

print(f'step_rewards {total_reward}')
clipped_reward = torch.log(torch.tensor(total_reward)).clip(-20)
print(f'clipped_reward {clipped_reward}')

map_info, _ = load_graph_files(map_lookup="S")
col_map = ["gold"] * len(map_info.n_info)

def display_graph(index):
    fig, ax = plt.subplots(figsize=(8, 6))
    cur_col_map = col_map[:]
    cur_col_map[red_path[index]-1] = "red"
    cur_col_map[blue_path[index]-1] = "blue"
    nx.draw_networkx(map_info.g_acs, pos=map_info.n_info, node_color=cur_col_map, edge_color="blue", arrows=True, ax=ax)
    ax.set_title(f"step rewards {step_rewards[index]}")
    plt.axis('off')
    plt.show()

# Create an interactive widget to display different graphs
slider = IntSlider(min=0, max=33, step=1, value=0, description='Graph Index')
interact(display_graph, index=slider)

AttributeError: 'GlowFigure8Squad' object has no attribute 'reset_state'

In [7]:
#visualize gat coordinate time

red_path = []
blue_path = []
step_rewards = []
total_reward = 0
gflowfigure8.reset_state_gat_coordinate_time()
for r in range(10):   
    step = gflowfigure8.step_gat_coordinate_time_deterministic(0)
    red_path.append(step['red_node'])
    blue_path.append(step['blue_node'])
    step_rewards.append(step['step_reward'])
    total_reward += step['step_reward']

print(f'step_rewards {total_reward}')
clipped_reward = torch.log(torch.tensor(total_reward)).clip(-20)
print(f'clipped_reward {clipped_reward}')

map_info, _ = load_graph_files(map_lookup="S")
col_map = ["gold"] * len(map_info.n_info)

def display_graph(index):
    fig, ax = plt.subplots(figsize=(8, 6))
    cur_col_map = col_map[:]
    cur_col_map[red_path[index]-1] = "red"
    cur_col_map[blue_path[index]-1] = "blue"
    nx.draw_networkx(map_info.g_acs, pos=map_info.n_info, node_color=cur_col_map, edge_color="blue", arrows=True, ax=ax)
    ax.set_title(f"step rewards {step_rewards[index]}")
    plt.axis('off')
    plt.show()

# Create an interactive widget to display different graphs
slider = IntSlider(min=0, max=9, step=1, value=0, description='Graph Index')
interact(display_graph, index=slider)

step_rewards 2
clipped_reward 0.6931471824645996


interactive(children=(IntSlider(value=0, description='Graph Index', max=9), Output()), _dom_classes=('widget-i…

<function __main__.display_graph(index)>

In [ ]:
red_path = [21, 27, 20, 14, 8, 7, 7, 8, 14, 20, 20, 20, 20, 20, 20, 20, 14, 8, 7, 7]
blue_path = [21, 27, 20, 14, 8,  7,  9,  10, 11, 16, 17, 22, 21, 27, 20, 26, 19, 25, 18, 12]

map_info, _ = load_graph_files(map_lookup="S")
col_map = ["gold"] * len(map_info.n_info)

def display_graph(index):
    fig, ax = plt.subplots(figsize=(8, 6))
    cur_col_map = col_map[:]
    cur_col_map[red_path[index]-1] = "red"
    cur_col_map[blue_path[index]-1] = "blue"
    nx.draw_networkx(map_info.g_acs, pos=map_info.n_info, node_color=cur_col_map, edge_color="blue", arrows=True, ax=ax)
    ax.set_title(f"step rewards {step_rewards[index]}")
    plt.axis('off')
    plt.show()

# Create an interactive widget to display different graphs
slider = IntSlider(min=0, max=9, step=1, value=0, description='Graph Index')
interact(display_graph, index=slider)